In [13]:
import stanza
from Loading_Json import *

import pandas as pd

df = pd.read_excel("./Exports/export_excel.xlsx")
df

,id,titre,date,year,legislature,Exposé des motifs,Communiqué de presse,Productions
0,JORFDOLE000025417300,LOI n° 2012-346 du 12 mars 2012 relative aux m...,Tue Mar 13 01:00:00 2012,2012,XIIIème législature,NaN,NaN,4
1,JORFDOLE000031600244,LOI n° 2015-1705 du 21 décembre 2015 autorisan...,Thu Dec 24 01:00:00 2015,2015,XIVème législature,La France a signé le 18 décembre 2008 à Paris...,NaN,3
2,JORFDOLE000026453342,LOI n° 2012-1432 du 21 décembre 2012 relative ...,Mon Dec 24 01:00:00 2012,2012,XIVème législature,"NOR : INTX1232040L Depuis 1986, la France s'e...",NaN,6
3,JORFDOLE000043266698,LOI n° 2021-725 du 8 juin 2021 visant à sécuri...,Wed Jun 9 02:00:00 2021,2021,XVème législature,NaN,NaN,4
4,JORFDOLE000023856350,LOI n° 2011-939 du 10 août 2011 sur la partici...,Thu Aug 11 02:00:00 2011,2011,XIIIème législature,Le présent projet de loi poursuit deux object...,NaN,6
...,...,...,...,...,...,...,...,...
740,JORFDOLE000030705573,LOI organique n° 2015-911 du 24 juillet 2015 r...,Sun Jul 26 02:00:00 2015,2015,XIVème législature,NaN,NaN,4
741,JORFDOLE000030641220,LOI n° 2015-957 du 3 août 2015 de règlement du...,Tue Aug 4 02:00:00 2015,2015,XIVème législature,NaN,Le ministre des finances et des comptes public...,0
742,JORFDOLE000018433118,LOI n° 2008-1091 du 28 octobre 2008 relative à...,Wed Oct 29 01:00:00 2008,2008,XIIIème législature,ExposÉ des motifs Le projet de loi qui vous e...,NaN,4
743,JORFDOLE000023273651,LOI organique n° 2014-1392 du 24 novembre 2014...,Tue Nov 25 01:00:00 2014,2014,XIVème législature,Représentant de la Nation et garant du foncti...,NaN,4


In [2]:
nlp = stanza.Pipeline('fr')

2021-12-07 17:07:32 INFO: Loading these models for language: fr (French):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |
| ner       | wikiner |

2021-12-07 17:07:32 INFO: Use device: cpu
2021-12-07 17:07:32 INFO: Loading: tokenize
2021-12-07 17:07:32 INFO: Loading: mwt
2021-12-07 17:07:32 INFO: Loading: pos
2021-12-07 17:07:33 INFO: Loading: lemma
2021-12-07 17:07:33 INFO: Loading: depparse
2021-12-07 17:07:34 INFO: Loading: ner
2021-12-07 17:07:39 INFO: Done loading processors!


In [3]:
import string

with open("./stopwords/Stop-words-french.txt", "r", encoding='utf8') as f:
    stopW1 = [line.rstrip("\n") for line in f.readlines()]
with open("./stopwords/stopword.txt", "r", encoding='utf8') as f:
    stopW2 = [line.rstrip("\n") for line in f.readlines()]
stopW = stopW1 + stopW2 #+ stopwords.words("french")
stopW.extend(string.punctuation)
stopW.extend(['’', '«', '»', '…', '—','».', '»,', '---', '.—', '…—', 'ans', 'an'])

In [22]:
import re
import nltk
import stanza
import time

def Lemma(df, tmp, i):

    start = time.time()

    com = df["Communiqué de presse"].loc[i]
    doc = df["Exposé des motifs"].loc[i]
    if pd.isnull(doc) and pd.isnull(com):
        print("none ", i-len(df))
    elif pd.isnull(doc) and pd.isnull(com) == False:
        com = re.sub(r'[^A-Za-zÀ-ÿ-’\' ]+', '', com)
    #    tok_com = nltk.wordpunct_tokenize(com)
        docu = nlp(com)
        docu = [word.lemma for sent in docu.sentences for word in sent.words]
        nostp_tok_com = [w for w in docu if w.lower() not in stopW]
        tmp["preprocessed_xpo"].loc[i] = nostp_tok_com
    else:
        doc = re.sub(r'[^A-Za-zÀ-ÿ-’\' ]+', '', doc)
    #    tok_doc = nltk.wordpunct_tokenize(doc)
        docu = nlp(doc)
        docu = [word.lemma for sent in docu.sentences for word in sent.words]
        nostp_tok_doc = [w for w in docu if w.lower() not in stopW]
        tmp["preprocessed_xpo"].loc[i] = nostp_tok_doc
    
    end = time.time()
    print(f'Temps d\'exécution : {(end-start):.2}s')
    return(tmp)

In [23]:
tempp = pd.DataFrame()
tempp["preprocessed_xpo"] = ["" for i in range(len(df))]
for i in range(len(df)):
    tmp = Lemma(df, tempp, i)

print(test.loc[1])

none  -745
Temps d'exécution : 0.00056s


/opt/anaconda3/lib/python3.8/site-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


Temps d'exécution : 4e+01s
Temps d'exécution : 2.5e+01s
none  -742
Temps d'exécution : 0.005s
Temps d'exécution : 6.1e+01s
none  -740
Temps d'exécution : 0.003s
Temps d'exécution : 7.7e+01s
Temps d'exécution : 8.9s
Temps d'exécution : 6e+01s
Temps d'exécution : 7.9e+01s
Temps d'exécution : 2.3e+01s
Temps d'exécution : 9e+01s
Temps d'exécution : 2.8e+01s
Temps d'exécution : 3.4e+01s
none  -731
Temps d'exécution : 0.00049s
none  -730
Temps d'exécution : 0.00099s
Temps d'exécution : 1.1e+01s
none  -728
Temps d'exécution : 0.00014s
none  -727
Temps d'exécution : 0.00012s
Temps d'exécution : 8.6e+01s
Temps d'exécution : 6.4e+01s
Temps d'exécution : 6.7e+01s
none  -723
Temps d'exécution : 0.0046s
Temps d'exécution : 7.3e+01s
none  -721
Temps d'exécution : 0.00057s
Temps d'exécution : 1.3e+01s
Temps d'exécution : 1e+01s
Temps d'exécution : 7.9e+01s
Temps d'exécution : 7.6s
Temps d'exécution : 1.8e+01s
Temps d'exécution : 7.3s
Temps d'exécution : 3.3e+01s
none  -713
Temps d'exécution : 0.00017

Temps d'exécution : 1e+01s
Temps d'exécution : 6.4e+01s
Temps d'exécution : 1e+01s
Temps d'exécution : 1.7e+01s
Temps d'exécution : 6.1e+01s
Temps d'exécution : 1e+01s
Temps d'exécution : 5.2s
Temps d'exécution : 9.2s
none  -480
Temps d'exécution : 0.00016s
none  -479
Temps d'exécution : 9.9e-05s
Temps d'exécution : 2.6e+01s
none  -477
Temps d'exécution : 0.00045s
none  -476
Temps d'exécution : 0.00014s
none  -475
Temps d'exécution : 0.00014s
Temps d'exécution : 6.7e+01s
Temps d'exécution : 2.9e+01s
Temps d'exécution : 2.4e+01s
none  -471
Temps d'exécution : 0.00016s
none  -470
Temps d'exécution : 0.00015s
Temps d'exécution : 5.1s
Temps d'exécution : 8.8s
Temps d'exécution : 3.5s
none  -466
Temps d'exécution : 0.0002s
none  -465
Temps d'exécution : 0.00025s
Temps d'exécution : 4.6e+01s
none  -463
Temps d'exécution : 0.0031s
Temps d'exécution : 1.3e+01s
Temps d'exécution : 1.4e+01s
none  -460
Temps d'exécution : 0.00014s
none  -459
Temps d'exécution : 0.00012s
Temps d'exécution : 1.1e+0

Temps d'exécution : 5.5s
Temps d'exécution : 6.2e+01s
Temps d'exécution : 4.7e+01s
Temps d'exécution : 1.5e+01s
Temps d'exécution : 1e+01s
none  -228
Temps d'exécution : 0.00022s
Temps d'exécution : 7e+01s
none  -226
Temps d'exécution : 0.00062s
Temps d'exécution : 1.2e+01s
Temps d'exécution : 6.5e+01s
none  -223
Temps d'exécution : 0.00097s
none  -222
Temps d'exécution : 0.00014s
Temps d'exécution : 5.2e+01s
Temps d'exécution : 6.4e+01s
Temps d'exécution : 2e+01s
Temps d'exécution : 3.8s
none  -217
Temps d'exécution : 0.00014s
Temps d'exécution : 6.8e+01s
Temps d'exécution : 4.9s
Temps d'exécution : 3.5s
Temps d'exécution : 8.5s
Temps d'exécution : 3.5e+01s
Temps d'exécution : 6.1e+01s
Temps d'exécution : 1.4e+01s
Temps d'exécution : 6.4e+01s
Temps d'exécution : 4.5e+01s
Temps d'exécution : 6.9s
none  -206
Temps d'exécution : 0.00041s
none  -205
Temps d'exécution : 0.0001s
none  -204
Temps d'exécution : 7.4e-05s
Temps d'exécution : 4.4s
none  -202
Temps d'exécution : 0.00019s
none  -2

In [330]:
tempp

' organique  2015-911 du 24 juillet 2015 relative à la nomination du président de la Commission nationale de contrôle des techniques de renseignement'

In [331]:
df = pd.read_excel("./Exports/export_excel.xlsx")
df = pd.concat([df, tempp], axis=1)
df.loc[1]

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

In [30]:
#df.to_excel("./Exports/final_export.xlsx")